In [ ]:
import torch
import numpy as np
import pandas as pd
import json

from src.model import build_model
from src.utils import convert_tensor, evaluate_model

In [ ]:
with open('./data/hyperparameter.json') as fp:
    hparam = json.load(fp)

In [ ]:
test_data = pd.read_pickle('./savepoints/0/test_data.pkl')
x = test_data[0]
y = test_data[1]

In [ ]:
SS_mat = pd.read_pickle('./data/structural_similarity_matrix.pkl')
TS_mat = pd.read_pickle('./data/target_similarity_matrix.pkl')
GS_mat = pd.read_pickle('./data/GO_similarity_matrix.pkl')

In [ ]:
mlb = pd.read_pickle('./data/mlb.pkl')
idx2label = pd.read_pickle('./data/idx2label.pkl')

In [ ]:
SS, TS, GS, y = convert_tensor(x, y, SS_mat, TS_mat, GS_mat, mlb, idx2label)
y = y.int().numpy()

In [ ]:
model = build_model(hparam)
model.load_model('./savepoints/0/model_checkpoint')
model.eval()

In [ ]:
with torch.no_grad():
    _, _, _, pred = model(SS, TS, GS)
    pred = torch.sigmoid(pred) > 0.5    
    pred = pred.int().numpy()

In [ ]:
acc, ma_rc, ma_pc, mi_rc, mi_pc = evaluate_model(y, pred)

In [ ]:
print(f'Accuracy: {acc}\nMacro recall: {ma_rc}\nMacro_precision: {ma_pc}\nMicro_recall: {mi_rc}\nMicro_precision: {mi_pc}')